In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
import datetime

effis = gpd.read_file('BRUTOS/EFFIS/modis.ba.poly.shp')
effis = effis[effis.COUNTRY == 'ES']
effis['year'] = effis.apply(lambda x: x.FIREDATE.split('-')[0], axis=1)
effis = effis[effis.year == '2022']
effis['SUP'] = effis.apply(lambda x: int(x.AREA_HA), axis=1)
gif = effis[effis.SUP > 500]
gif['fecha'] = gif.apply(lambda x: x.FIREDATE.split(' ')[0], axis=1)
gif['f'] = gif.apply(lambda x: datetime.datetime.strptime(x.fecha, '%Y-%m-%d'), axis=1)
gif['fecha_fin'] = gif.apply(lambda x: x.LASTUPDATE.split(' ')[0], axis=1)
gif['fecha_fin'] = gif.apply(lambda x: datetime.datetime.strptime(x.fecha_fin, '%Y-%m-%d'), axis=1)
gif = gif[gif.f >= datetime.datetime.strptime('2022-06-01', '%Y-%m-%d')]
display(gif)

In [ ]:
comunidades = gpd.read_file('../navarra/BRUTOS/COMUNIDADES/SHP_ETRS89/recintos_autonomicas_inspire_peninbal_etrs89/recintos_autonomicas_inspire_peninbal_etrs89.shp')

In [ ]:
inicio = datetime.datetime.strptime('2022-06-01', '%Y-%m-%d')
for i in range(154):
    fecha = inicio + datetime.timedelta(days=i)
    inc = gif[gif.f <= fecha]

    figure = plt.figure(figsize=(14,10))
    proj = ccrs.PlateCarree()
    ax = plt.subplot(projection=proj)

    minx, miny, maxx, maxy = comunidades.total_bounds
    ax.set_extent([minx, maxx, miny, maxy], crs=proj)
    comunidades.plot(ax=ax, edgecolor='black', facecolor=(0,0,1,0))
    for j, row in inc.iterrows():
        if row.fecha_fin >= fecha:
            ax.add_geometries([row.geometry], facecolor=(1,0,0,1), crs=proj)
            ax.plot(row.geometry.centroid.x, row.geometry.centroid.y, 'r', marker='.', markersize=50)
        else:
            ax.add_geometries([row.geometry], facecolor=(0,0,0,1), crs=proj)
    plt.savefig('MAPAS/mapa_{}.png'.format(fecha.strftime('%Y%m%d')), bbox_inches='tight')